In [0]:
import os

In [2]:
cd /content/

/content


In [0]:
!mkdir train
!mkdir test
!mkdir checkpoint

In [4]:
os.listdir()

['.config',
 'train',
 'sample_submission.csv',
 'test',
 'checkpoint',
 'test.zip',
 'train.zip',
 'sample_data']

In [5]:
!unzip test.zip -d test

Streaming output truncated to the last 5000 lines.
  inflating: test/Test/1576.jpg      
  inflating: test/Test/15762.jpg     
  inflating: test/Test/15765.jpg     
  inflating: test/Test/15768.jpg     
  inflating: test/Test/15770.jpg     
  inflating: test/Test/15773.jpg     
  inflating: test/Test/15778.jpg     
  inflating: test/Test/15780.jpg     
  inflating: test/Test/15784.jpg     
  inflating: test/Test/15803.jpg     
  inflating: test/Test/15805.jpg     
  inflating: test/Test/15806.jpg     
  inflating: test/Test/15808.jpg     
  inflating: test/Test/1581.jpg      
  inflating: test/Test/15810.jpg     
  inflating: test/Test/15811.jpg     
  inflating: test/Test/15812.jpg     
  inflating: test/Test/15813.jpg     
  inflating: test/Test/15814.jpg     
  inflating: test/Test/15815.jpg     
  inflating: test/Test/15824.jpg     
  inflating: test/Test/15825.jpg     
  inflating: test/Test/15827.jpg     
  inflating: test/Test/15831.jpg     
  inflating: test/Test/15833.jpg     

In [6]:
!unzip train.zip -d train

Streaming output truncated to the last 5000 lines.
  inflating: train/Train/4047.jpg    
  inflating: train/Train/4049.jpg    
  inflating: train/Train/405.jpg     
  inflating: train/Train/4051.jpg    
  inflating: train/Train/4052.jpg    
  inflating: train/Train/4053.jpg    
  inflating: train/Train/4055.jpg    
  inflating: train/Train/4056.jpg    
  inflating: train/Train/4057.jpg    
  inflating: train/Train/4058.jpg    
  inflating: train/Train/4059.jpg    
  inflating: train/Train/406.jpg     
  inflating: train/Train/4060.jpg    
  inflating: train/Train/4061.jpg    
  inflating: train/Train/4062.jpg    
  inflating: train/Train/4065.jpg    
  inflating: train/Train/4066.jpg    
  inflating: train/Train/4068.jpg    
  inflating: train/Train/4069.jpg    
  inflating: train/Train/407.jpg     
  inflating: train/Train/4070.jpg    
  inflating: train/Train/4072.jpg    
  inflating: train/Train/4073.jpg    
  inflating: train/Train/4076.jpg    
  inflating: train/Train/4078.jpg    

In [0]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
import numpy as np
import time
import pandas as pd
from PIL import Image
from skimage import io
import glob
from collections import OrderedDict
from tqdm import tqdm_notebook as tqdm
from torch.utils.data import Dataset, DataLoader

import torch
from torch import nn
from torch.utils import data
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision
from torchvision import datasets, transforms, models


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

OPTIMIZERS

ranger implementation error


In [0]:
!git clone https://github.com/lessw2020/Ranger-Deep-Learning-Optimizer
!cd Ranger-Deep-Learning-Optimizer


In [0]:
!pip install ranger


used 0ver9000

In [0]:
import torch, math
from torch.optim.optimizer import Optimizer

# RAdam + LARS
class Ralamb(Optimizer):

    def __init__(self, params, lr=1e-3, betas=(0.9, 0.999), eps=1e-8, weight_decay=0):
        defaults = dict(lr=lr, betas=betas, eps=eps, weight_decay=weight_decay)
        self.buffer = [[None, None, None] for ind in range(10)]
        super(Ralamb, self).__init__(params, defaults)

    def __setstate__(self, state):
        super(Ralamb, self).__setstate__(state)

    def step(self, closure=None):

        loss = None
        if closure is not None:
            loss = closure()

        for group in self.param_groups:

            for p in group['params']:
                if p.grad is None:
                    continue
                grad = p.grad.data.float()
                if grad.is_sparse:
                    raise RuntimeError('Ralamb does not support sparse gradients')

                p_data_fp32 = p.data.float()

                state = self.state[p]

                if len(state) == 0:
                    state['step'] = 0
                    state['exp_avg'] = torch.zeros_like(p_data_fp32)
                    state['exp_avg_sq'] = torch.zeros_like(p_data_fp32)
                else:
                    state['exp_avg'] = state['exp_avg'].type_as(p_data_fp32)
                    state['exp_avg_sq'] = state['exp_avg_sq'].type_as(p_data_fp32)

                exp_avg, exp_avg_sq = state['exp_avg'], state['exp_avg_sq']
                beta1, beta2 = group['betas']

                # Decay the first and second moment running average coefficient
                # m_t
                exp_avg.mul_(beta1).add_(1 - beta1, grad)
                # v_t
                exp_avg_sq.mul_(beta2).addcmul_(1 - beta2, grad, grad)

                state['step'] += 1
                buffered = self.buffer[int(state['step'] % 10)]

                if state['step'] == buffered[0]:
                    N_sma, radam_step_size = buffered[1], buffered[2]
                else:
                    buffered[0] = state['step']
                    beta2_t = beta2 ** state['step']
                    N_sma_max = 2 / (1 - beta2) - 1
                    N_sma = N_sma_max - 2 * state['step'] * beta2_t / (1 - beta2_t)
                    buffered[1] = N_sma

                    # more conservative since it's an approximated value
                    if N_sma >= 5:
                        radam_step_size = math.sqrt((1 - beta2_t) * (N_sma - 4) / (N_sma_max - 4) * (N_sma - 2) / N_sma * N_sma_max / (N_sma_max - 2)) / (1 - beta1 ** state['step'])
                    else:
                        radam_step_size = 1.0 / (1 - beta1 ** state['step'])
                    buffered[2] = radam_step_size

                if group['weight_decay'] != 0:
                    p_data_fp32.add_(-group['weight_decay'] * group['lr'], p_data_fp32)

                # more conservative since it's an approximated value
                radam_step = p_data_fp32.clone()
                if N_sma >= 5:
                    denom = exp_avg_sq.sqrt().add_(group['eps'])
                    radam_step.addcdiv_(-radam_step_size * group['lr'], exp_avg, denom)
                else:
                    radam_step.add_(-radam_step_size * group['lr'], exp_avg)

                radam_norm = radam_step.pow(2).sum().sqrt()
                weight_norm = p.data.pow(2).sum().sqrt().clamp(0, 10)
                if weight_norm == 0 or radam_norm == 0:
                    trust_ratio = 1
                else:
                    trust_ratio = weight_norm / radam_norm

                state['weight_norm'] = weight_norm
                state['adam_norm'] = radam_norm
                state['trust_ratio'] = trust_ratio

                if N_sma >= 5:
                    p_data_fp32.addcdiv_(-radam_step_size * group['lr'] * trust_ratio, exp_avg, denom)
                else:
                    p_data_fp32.add_(-radam_step_size * group['lr'] * trust_ratio, exp_avg)

                p.data.copy_(p_data_fp32)

        return loss

# Lookahead implementation from https://github.com/rwightman/pytorch-image-models/blob/master/timm/optim/lookahead.py

""" Lookahead Optimizer Wrapper.
Implementation modified from: https://github.com/alphadl/lookahead.pytorch
Paper: `Lookahead Optimizer: k steps forward, 1 step back` - https://arxiv.org/abs/1907.08610
"""
import torch
from torch.optim.optimizer import Optimizer
from collections import defaultdict

class Lookahead(Optimizer):
    def __init__(self, base_optimizer, alpha=0.5, k=6):
        if not 0.0 <= alpha <= 1.0:
            raise ValueError(f'Invalid slow update rate: {alpha}')
        if not 1 <= k:
            raise ValueError(f'Invalid lookahead steps: {k}')
        defaults = dict(lookahead_alpha=alpha, lookahead_k=k, lookahead_step=0)
        self.base_optimizer = base_optimizer
        self.param_groups = self.base_optimizer.param_groups
        self.defaults = base_optimizer.defaults
        self.defaults.update(defaults)
        self.state = defaultdict(dict)
        # manually add our defaults to the param groups
        for name, default in defaults.items():
            for group in self.param_groups:
                group.setdefault(name, default)

    def update_slow(self, group):
        for fast_p in group["params"]:
            if fast_p.grad is None:
                continue
            param_state = self.state[fast_p]
            if 'slow_buffer' not in param_state:
                param_state['slow_buffer'] = torch.empty_like(fast_p.data)
                param_state['slow_buffer'].copy_(fast_p.data)
            slow = param_state['slow_buffer']
            slow.add_(group['lookahead_alpha'], fast_p.data - slow)
            fast_p.data.copy_(slow)

    def sync_lookahead(self):
        for group in self.param_groups:
            self.update_slow(group)

    def step(self, closure=None):
        # print(self.k)
        #assert id(self.param_groups) == id(self.base_optimizer.param_groups)
        loss = self.base_optimizer.step(closure)
        for group in self.param_groups:
            group['lookahead_step'] += 1
            if group['lookahead_step'] % group['lookahead_k'] == 0:
                self.update_slow(group)
        return loss

    def state_dict(self):
        fast_state_dict = self.base_optimizer.state_dict()
        slow_state = {
            (id(k) if isinstance(k, torch.Tensor) else k): v
            for k, v in self.state.items()
        }
        fast_state = fast_state_dict['state']
        param_groups = fast_state_dict['param_groups']
        return {
            'state': fast_state,
            'slow_state': slow_state,
            'param_groups': param_groups,
        }

    def load_state_dict(self, state_dict):
        fast_state_dict = {
            'state': state_dict['state'],
            'param_groups': state_dict['param_groups'],
        }
        self.base_optimizer.load_state_dict(fast_state_dict)

        # We want to restore the slow state, but share param_groups reference
        # with base_optimizer. This is a bit redundant but least code
        slow_state_new = False
        if 'slow_state' not in state_dict:
            print('Loading state_dict from optimizer without Lookahead applied.')
            state_dict['slow_state'] = defaultdict(dict)
            slow_state_new = True
        slow_state_dict = {
            'state': state_dict['slow_state'],
            'param_groups': state_dict['param_groups'],  # this is pointless but saves code
        }
        super(Lookahead, self).load_state_dict(slow_state_dict)
        self.param_groups = self.base_optimizer.param_groups  # make both ref same container
        if slow_state_new:
            # reapply defaults to catch missing lookahead specific ones
            for name, default in self.defaults.items():
                for group in self.param_groups:
                    group.setdefault(name, default)

def LookaheadAdam(params, alpha=0.5, k=6, *args, **kwargs):
     adam = Adam(params, *args, **kwargs)
     return Lookahead(adam, alpha, k)


# RAdam + LARS + LookAHead

# Lookahead implementation from https://github.com/lonePatient/lookahead_pytorch/blob/master/optimizer.py
# RAdam + LARS implementation from https://gist.github.com/redknightlois/c4023d393eb8f92bb44b2ab582d7ec20

def Over9000(params, alpha=0.5, k=5, *args, **kwargs):
     ralamb = Ralamb(params, *args, **kwargs)
     return Lookahead(ralamb, alpha, k)

RangerLars = Over9000 

In [0]:
sample_submission=pd.read_csv('sample_submission.csv')
sample_submission.head()

In [0]:
df_train=pd.read_csv('train/train.csv')
df1 = df_train['Class']
df2 = df_train["ID"]
df1 = pd.get_dummies(df1)
df = pd.concat([df2,df1], axis=1)
df.head()

In [0]:
columns_titles = ['ID',"YOUNG","MIDDLE",'OLD']

df=df.reindex(columns=columns_titles)
df.head()

In [0]:
train = df.sample(frac=0.8)
valid = df[~df['ID'].isin(train['ID'])]

In [0]:
print(train.head())
print(train.shape)

In [0]:

from albumentations.pytorch import ToTensor
import albumentations as albu


DATALOADER

removed randomcrop as too much noise

In [0]:
class train_Age(Dataset):
    
    def __init__(self,df):
        
        self.df = df
        self.transforms = albu.Compose([
               albu.SmallestMaxSize(256),
               
               albu.HorizontalFlip(),
               albu.Blur(blur_limit=5),
               albu.RGBShift(),
               albu.RandomBrightness(),
               albu.RandomContrast(),
               albu.GaussNoise(),
               albu.Cutout(4),
               albu.Rotate(limit=30),
               albu.Resize(256,256),
               albu.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)),
                ToTensor()
            ])
       
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        label = self.df.iloc[index,1:]
        label = label.to_numpy()
        label = np.argmax(label)
        name = self.df.iloc[index,0]
        
        
        path = 'train/Train/' + str(name)
        img = plt.imread(path)
        img = self.transforms(image = np.array(img))
        img = img['image']
        
        return (img,label)



class test_Age(Dataset):
    
    def __init__(self,df,phase='test'):
        
        self.df = df
        self.phase=phase
        self.transforms = albu.Compose([
                albu.Resize(256,256),
                albu.Normalize((0.485, 0.456, 0.406),(0.229, 0.224, 0.225)),
                ToTensor()
            ])
        
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self,index):
        label = self.df.iloc[index,1:]
        label = label.to_numpy()
        label = np.argmax(label)
        name = self.df.iloc[index,0]
        
        if self.phase=='test':
          path='test/Test/' + str(name)
        elif self.phase=='val':
          path = 'train/Train/' + str(name)
        img = plt.imread(path)
        img = self.transforms(image = np.array(img))
        img = img['image']
        
        return img,label

In [0]:
traindata = train_Age(train)
valdata=test_Age(valid,phase='val')

In [0]:
trainloader = DataLoader(traindata,batch_size=16)
valloader=DataLoader(valdata,batch_size=8)



In [0]:
dataiter = iter(trainloader)
img,label = dataiter.next()
img.shape

In [0]:
mean = np.array([0.485, 0.456, 0.406])
std = np.array([0.229, 0.224, 0.225])
def im_show(img):
    npimg = img.numpy().transpose((1,2,0))*std + mean
    npimg = np.clip(npimg, 0., 1.)
    plt.imshow(npimg)
fig = plt.figure(figsize=(18,5))

for i in np.arange(16):
    ax = fig.add_subplot(2,8,i+1,xticks=[],yticks=[])
    im_show(img[i])
    

TRAINING

trained_model with weights saved 

In [0]:
def train_model(dataloders, model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()
    print('start time: {:10f} minutes'.format((time.time())/60))
    use_gpu = torch.cuda.is_available()
    best_model_wts = model.state_dict()
    best_acc = 0.0
    start_time = time.time()
    dataset_sizes = {'train': len(dataloders['train'].dataset), 
                     'valid': len(dataloders['valid'].dataset)}
    valid_epoch_loss=999
    for epoch in range(num_epochs):
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step(valid_epoch_loss)
                model.train(True)
            else:
                model.train(False)

            running_loss = 0.0
            running_corrects = 0

            for inputs, labels in dataloders[phase]:
                if use_gpu:
                    inputs, labels = Variable(inputs.cuda()), Variable(labels.cuda())
                else:
                    inputs, labels = Variable(inputs), Variable(labels)

                optimizer.zero_grad()

                outputs = model(inputs)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, labels)

                if phase == 'train':
                    loss.backward()
                    optimizer.step()

                running_loss += (loss.data).item()*inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
            
            if phase == 'train':
                train_epoch_loss = float(running_loss) / dataset_sizes[phase]
                train_epoch_acc = float(running_corrects) / dataset_sizes[phase]
            else:
                valid_epoch_loss = float(running_loss) / dataset_sizes[phase]
                valid_epoch_acc = float(running_corrects) / dataset_sizes[phase]
                
            if phase == 'valid' and valid_epoch_acc > best_acc:
                best_acc = valid_epoch_acc
                best_model_wts = model.state_dict()
                torch.save(model.state_dict(),'./checkpoint/current_checkpoint.pt')
        print('Training time: {:10f} minutes'.format((time.time()-start_time)/60))
        print('Epoch [{}/{}] train loss: {:.4f} acc: {:.4f} ' 
              'valid loss: {:.4f} acc: {:.4f}'.format(
                epoch, num_epochs - 1,
                train_epoch_loss, train_epoch_acc, 
                valid_epoch_loss, valid_epoch_acc))
            
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model

In [0]:
!pip install pytorch_ranger
from pytorch_ranger import Ranger

used efficientnetb5

In [23]:
!pip install efficientnet_pytorch
from efficientnet_pytorch import EfficientNet

  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.6.3-cp36-none-any.whl size=12422 sha256=673f42f59907d6fef74720173d65d55abc30a330a8f62ef0eac0c37991185264
  Stored in directory: /root/.cache/pip/wheels/42/1e/a9/2a578ba9ad04e776e80bf0f70d8a7f4c29ec0718b92d8f6ccd
Successfully built efficientnet-pytorch


In [0]:
model = EfficientNet.from_pretrained('efficientnet-b3')
# freeze all model parameters
for param in model.parameters():
    param.requires_grad = False

in_features = model._fc.in_features
classifier =nn.Linear(in_features, 3)
model._fc=classifier

if torch.cuda.is_available():
    model = model.cuda()

criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
optimizer =Ranger(model.parameters(), lr=2e-3, weight_decay=1e-3)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=3, verbose=False, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

dataloaders = {'train':trainloader, 'valid':valloader}    

In [0]:
start_time = time.time()
model = train_model( dataloaders,model, criterion, optimizer, lr_scheduler, num_epochs=5)
print('Training time: {:10f} minutes'.format((time.time()-start_time)/60))

In [36]:
model=EfficientNet.from_pretrained('efficientnet-b3')
child_counter = 0
for child in model.children():
    print(" child", child_counter, "is -")
    print(child)
    child_counter += 1


Loaded pretrained weights for efficientnet-b3
 child 0 is -
Conv2dStaticSamePadding(
  3, 40, kernel_size=(3, 3), stride=(2, 2), bias=False
  (static_padding): ZeroPad2d(padding=(0, 1, 0, 1), value=0.0)
)
 child 1 is -
BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
 child 2 is -
ModuleList(
  (0): MBConvBlock(
    (_depthwise_conv): Conv2dStaticSamePadding(
      40, 40, kernel_size=(3, 3), stride=[1, 1], groups=40, bias=False
      (static_padding): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
    )
    (_bn1): BatchNorm2d(40, eps=0.001, momentum=0.010000000000000009, affine=True, track_running_stats=True)
    (_se_reduce): Conv2dStaticSamePadding(
      40, 10, kernel_size=(1, 1), stride=(1, 1)
      (static_padding): Identity()
    )
    (_se_expand): Conv2dStaticSamePadding(
      10, 40, kernel_size=(1, 1), stride=(1, 1)
      (static_padding): Identity()
    )
    (_project_conv): Conv2dStaticSamePadding(
      40, 24, kernel_size=(

TRAINED LAST LAYER- 5epochs  (4 should be better)

In [43]:
model = EfficientNet.from_pretrained('efficientnet-b3')
# freeze all model parameters
for param in model.parameters():
    param.requires_grad = True

# new final layer with 16 classes
in_features = model._fc.in_features
classifier =nn.Linear(in_features, 3)

child_counter = 0
for child in model.children():
    if child_counter < 2:
        print("child ",child_counter," was frozen")
        for param in child.parameters():
            param.requires_grad = False
    elif child_counter == 2:
        children_of_child_counter = 0
        for children_of_child in child.children():
            if children_of_child_counter < 25:
                for param in children_of_child.parameters():
                    param.requires_grad = False
                print('child ', children_of_child_counter, 'of child',child_counter,' was frozen')
            else:
                print('child ', children_of_child_counter, 'of child',child_counter,' was not frozen')
            children_of_child_counter += 1

    else:
        print("child ",child_counter," was not frozen")
    child_counter += 1

model._fc=classifier
model.load_state_dict(torch.load('./checkpoint/current_checkpoint.pt'),strict=True)
if torch.cuda.is_available():
    model = model.cuda()

criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
optimizer =Ranger(model.parameters(), lr=2e-3, weight_decay=1e-3)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=3, verbose=False, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

dataloaders = {'train':trainloader, 'valid':valloader}

Loaded pretrained weights for efficientnet-b3
child  0  was frozen
child  1  was frozen
child  0 of child 2  was frozen
child  1 of child 2  was frozen
child  2 of child 2  was frozen
child  3 of child 2  was frozen
child  4 of child 2  was frozen
child  5 of child 2  was frozen
child  6 of child 2  was frozen
child  7 of child 2  was frozen
child  8 of child 2  was frozen
child  9 of child 2  was frozen
child  10 of child 2  was frozen
child  11 of child 2  was frozen
child  12 of child 2  was frozen
child  13 of child 2  was frozen
child  14 of child 2  was frozen
child  15 of child 2  was frozen
child  16 of child 2  was frozen
child  17 of child 2  was frozen
child  18 of child 2  was frozen
child  19 of child 2  was frozen
child  20 of child 2  was frozen
child  21 of child 2  was frozen
child  22 of child 2  was frozen
child  23 of child 2  was frozen
child  24 of child 2  was frozen
child  25 of child 2  was not frozen
child  3  was not frozen
child  4  was not frozen
child  5  

In [44]:
from tqdm import tqdm
from tqdm import tqdm_notebook
start_time = time.time()
model = train_model( dataloaders,model, criterion, optimizer, lr_scheduler, num_epochs=5)
print('Training time: {:10f} minutes'.format((time.time()-start_time)/60))

start time: 26507333.947227 minutes
Training time:   7.884448 minutes
Epoch [0/4] train loss: 0.4974 acc: 0.7913 valid loss: 0.4957 acc: 0.7958
Training time:  15.758236 minutes
Epoch [1/4] train loss: 0.4845 acc: 0.7990 valid loss: 0.4560 acc: 0.8106
Training time:  23.675905 minutes
Epoch [2/4] train loss: 0.4657 acc: 0.8095 valid loss: 0.5263 acc: 0.7918
Training time:  31.596233 minutes
Epoch [3/4] train loss: 0.4518 acc: 0.8131 valid loss: 0.4532 acc: 0.8214
Training time:  39.469589 minutes
Epoch [4/4] train loss: 0.4369 acc: 0.8176 valid loss: 0.4489 acc: 0.8237
Best val Acc: 0.823662
Training time:  39.470496 minutes


TRAINED ALL LAYERS- 10 epochs  (will try more epochs later)

In [46]:
model = EfficientNet.from_pretrained('efficientnet-b3')

# freeze all model parameters
for param in model.parameters():
    param.requires_grad = True

# new final layer with 16 classes
in_features = model._fc.in_features
classifier =nn.Linear(in_features, 3)
                           
model._fc=classifier
model.load_state_dict(torch.load('./checkpoint/current_checkpoint.pt'),strict=True)

if torch.cuda.is_available():
    model = model.cuda()

criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
optimizer =Ranger(model.parameters(), lr=1e-3, weight_decay=1e-3)
lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=3, verbose=False, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

dataloaders = {'train':trainloader, 'valid':valloader}

Loaded pretrained weights for efficientnet-b3


In [0]:
start_time = time.time()
model = train_model( dataloaders,model, criterion, optimizer, lr_scheduler, num_epochs=10)
print('Training time: {:10f} minutes'.format((time.time()-start_time)/60))

start time: 26507373.655305 minutes
Training time:  15.197719 minutes
Epoch [0/9] train loss: 0.4504 acc: 0.8141 valid loss: 0.4722 acc: 0.8156
Training time:  30.375659 minutes
Epoch [1/9] train loss: 0.4261 acc: 0.8295 valid loss: 0.4010 acc: 0.8420


TRAINED LAST LAYER- 4 epochs

In [0]:
model = EfficientNet.from_pretrained('efficientnet-b5')

# freeze all model parameters
for param in model.parameters():
    param.requires_grad = False

# new final layer with 16 classes
in_features = model._fc.in_features
classifier =nn.Linear(in_features, 3)
                           
model._fc=classifier
model.load_state_dict(torch.load('./checkpoint/current_checkpoint.pt'),strict=True)

if torch.cuda.is_available():
    model = model.cuda()

criterion = torch.nn.CrossEntropyLoss()
#optimizer = torch.optim.AdamW(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08, weight_decay=0.01, amsgrad=False)
optimizer =Ranger(model.parameters(), lr=1e-3, weight_decay=1e-3)
exp_lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.3, patience=3, verbose=False, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)

dataloaders = {'train':trainloader, 'valid':valloader}

Loaded pretrained weights for efficientnet-b5


In [0]:
start_time = time.time()
model = train_model( dataloaders,model, criterion, optimizer, exp_lr_scheduler, num_epochs=4)
print('Training time: {:10f} minutes'.format((time.time()-start_time)/60))

start time: 26504578.484846 minutes
Training time:   5.290921 minutes
Epoch [0/3] train loss: 0.1093 acc: 0.9613 valid loss: 0.3290 acc: 0.9030
Training time:  10.567391 minutes
Epoch [1/3] train loss: 0.0992 acc: 0.9651 valid loss: 0.3190 acc: 0.9050
Training time:  15.797041 minutes
Epoch [2/3] train loss: 0.0907 acc: 0.9673 valid loss: 0.3197 acc: 0.9040
Training time:  21.075119 minutes
Epoch [3/3] train loss: 0.0883 acc: 0.9685 valid loss: 0.3236 acc: 0.9050
Best val Acc: 0.905049
Training time:  21.076473 minutes


In [0]:
output=sample_submission[['ID','Class']]

In [0]:
output.head()

,ID,Class
0,25321.jpg,MIDDLE
1,989.jpg,MIDDLE
2,19277.jpg,MIDDLE
3,13093.jpg,MIDDLE
4,5367.jpg,MIDDLE


In [0]:
testdata=test_Age(output)

In [0]:
testloader=DataLoader(testdata,batch_size=16)

In [0]:
def test_submission(model):
    since = time.time()
    sub_output = []
    model.train(False)
    for data in testloader:
        inputs,labels = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _,pred = torch.max(outputs.data,1)
        pred = pred.data.cpu().numpy()
        sub_output.append(pred.data)
    sub_output = np.concatenate(sub_output)
    output['Class'] = sub_output
    print()
    time_elapsed = time.time() - since
    print('Run complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))

In [0]:
test_submission(model)


Run complete in 0m 55s


In [0]:
output.head()

,ID,Class
0,25321.jpg,1
1,989.jpg,0
2,19277.jpg,1
3,13093.jpg,2
4,5367.jpg,1


In [0]:
submission=output[['ID','Class']]

In [0]:
submission=submission.replace(0,'YOUNG')
submission=submission.replace(1,'MIDDLE')
submission=submission.replace(2,'OLD')



In [0]:
submission.to_csv('SUBMISSION3(2).csv',index=False)